In [1]:
%load_ext sql

In [2]:
%sql sqlite:///C:\Users\User\Desktop\sql\chinook.db

In [60]:
%%sql
SELECT * 
FROM 
 sqlite_master

 * sqlite:///C:\Users\User\Desktop\sql\chinook.db
Done.


type,name,tbl_name,rootpage,sql
table,albums,albums,2,"CREATE TABLE ""albums"" ( [AlbumId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Title] NVARCHAR(160) NOT NULL, [ArtistId] INTEGER NOT NULL, FOREIGN KEY ([ArtistId]) REFERENCES ""artists"" ([ArtistId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,artists,artists,4,"CREATE TABLE ""artists"" ( [ArtistId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Name] NVARCHAR(120) )"
table,customers,customers,5,"CREATE TABLE ""customers"" ( [CustomerId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [FirstName] NVARCHAR(40) NOT NULL, [LastName] NVARCHAR(20) NOT NULL, [Company] NVARCHAR(80), [Address] NVARCHAR(70), [City] NVARCHAR(40), [State] NVARCHAR(40), [Country] NVARCHAR(40), [PostalCode] NVARCHAR(10), [Phone] NVARCHAR(24), [Fax] NVARCHAR(24), [Email] NVARCHAR(60) NOT NULL, [SupportRepId] INTEGER, FOREIGN KEY ([SupportRepId]) REFERENCES ""employees"" ([EmployeeId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
table,employees,employees,8,"CREATE TABLE ""employees"" ( [EmployeeId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [LastName] NVARCHAR(20) NOT NULL, [FirstName] NVARCHAR(20) NOT NULL, [Title] NVARCHAR(30), [ReportsTo] INTEGER, [BirthDate] DATETIME, [HireDate] DATETIME, [Address] NVARCHAR(70), [City] NVARCHAR(40), [State] NVARCHAR(40), [Country] NVARCHAR(40), [PostalCode] NVARCHAR(10), [Phone] NVARCHAR(24), [Fax] NVARCHAR(24), [Email] NVARCHAR(60), FOREIGN KEY ([ReportsTo]) REFERENCES ""employees"" ([EmployeeId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
table,genres,genres,10,"CREATE TABLE ""genres"" ( [GenreId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Name] NVARCHAR(120) )"
table,invoices,invoices,11,"CREATE TABLE ""invoices"" ( [InvoiceId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [CustomerId] INTEGER NOT NULL, [InvoiceDate] DATETIME NOT NULL, [BillingAddress] NVARCHAR(70), [BillingCity] NVARCHAR(40), [BillingState] NVARCHAR(40), [BillingCountry] NVARCHAR(40), [BillingPostalCode] NVARCHAR(10), [Total] NUMERIC(10,2) NOT NULL, FOREIGN KEY ([CustomerId]) REFERENCES ""customers"" ([CustomerId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
table,invoice_items,invoice_items,13,"CREATE TABLE ""invoice_items"" ( [InvoiceLineId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [InvoiceId] INTEGER NOT NULL, [TrackId] INTEGER NOT NULL, [UnitPrice] NUMERIC(10,2) NOT NULL, [Quantity] INTEGER NOT NULL, FOREIGN KEY ([InvoiceId]) REFERENCES ""invoices"" ([InvoiceId]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([TrackId]) REFERENCES ""tracks"" ([TrackId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
table,media_types,media_types,15,"CREATE TABLE ""media_types"" ( [MediaTypeId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Name] NVARCHAR(120) )"
table,playlists,playlists,16,"CREATE TABLE ""playlists"" ( [PlaylistId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Name] NVARCHAR(120) )"


In [62]:
%%sql
-- To view availables Tables
SELECT name from sqlite_master where type = "table"

 * sqlite:///C:\Users\User\Desktop\sql\chinook.db
Done.


name
albums
sqlite_sequence
artists
customers
employees
genres
invoices
invoice_items
media_types
playlists


In [26]:
%%sql
-- 1. Create a sales report showing total revenue per country per year
SELECT 
    c.Country as Country,
    SUM(CASE WHEN strftime("%Y", i.InvoiceDate)  = "2009" THEN round(i.Total, 2) ELSE 0 END) as Revenue_2009,
    SUM(CASE WHEN strftime("%Y", i.InvoiceDate)  = "2010" THEN i.Total ELSE 0 END) as Revenue_2010,
    SUM(CASE WHEN strftime("%Y", i.InvoiceDate) = "2011" THEN i.Total ELSE 0 END) as Revenue_2011
FROM 
    customers c
JOIN
    invoices i
ON
    c.CustomerId = i.CustomerId	

GROUP BY c.Country

LIMIT 5

 * sqlite:///C:\Users\User\Desktop\sql\chinook.db
Done.


Country,Revenue_2009,Revenue_2010,Revenue_2011
Argentina,0,11.88,0.99
Australia,11.88,0.99,1.98
Austria,1.98,27.77,0
Belgium,6.930000000000001,0,24.75
Brazil,37.62,41.6,19.8


In [27]:
%%sql
--  2. Find the top 5 employees (support reps) by total sales
SELECT 
    e.LastName || " "	|| e.FirstName as Employeee,
    sum(i.Total) as Total_Sales

FROM 
    employees e
JOIN 
    customers c
ON
    c.SupportRepId = e.EmployeeId	
JOIN 
    invoices i
ON
    c.CustomerId = i.CustomerId
GROUP BY c.CustomerId
ORDER BY Total DESC
LIMIT 5

 * sqlite:///C:\Users\User\Desktop\sql\chinook.db
Done.


Employeee,Total_Sales
Park Margaret,37.62
Johnson Steve,49.62
Park Margaret,37.62
Johnson Steve,37.62
Park Margaret,37.62


In [48]:
%%sql
--   3. Report: Revenue per genre, broken down by media type
SELECT 
    g.Name as Genre,
    m.Name as Media_type,
    round(sum(it.UnitPrice * it.Quantity),3) as Revenue
FROM 
    genres g
JOIN 
    tracks t
ON
    g.GenreId = t.GenreId
JOIN
    invoice_items it
ON 
    t.TrackId = it.TrackId
JOIN 
    media_types m
ON 
    t.MediaTypeId = m.MediaTypeId
GROUP BY 
     m.Name, g.Name
ORDER BY 
    Revenue DESC

LIMIT 5

 * sqlite:///C:\Users\User\Desktop\sql\chinook.db
Done.


Genre,Media_type,Revenue
Rock,MPEG audio file,765.27
Latin,MPEG audio file,381.15
Metal,MPEG audio file,261.36
Alternative & Punk,MPEG audio file,241.56
TV Shows,Protected MPEG-4 video file,93.53


In [57]:
%%sql
-- 4. Find the percentage contribution of each genre to total sales
with Genre_contribution as (
    SELECT 
        g.Name as Genre,
        sum(it.UnitPrice * it.Quantity) as Revenue
    FROM 
        genres g
    JOIN 
        tracks t
    ON
        g.GenreId = t.GenreId
    JOIN
        invoice_items it
    ON 
        t.TrackId = it.TrackId
    JOIN 
        invoices i
    ON 
        it.InvoiceId = i.InvoiceId
    GROUP BY
         g.Name
)
SELECT 
    Genre,
    round((Revenue / (SELECT sum(Revenue) FROM Genre_contribution))* 100, 3) || "%" as Percent_contribution
FROM
    Genre_contribution
LIMIT 5

 * sqlite:///C:\Users\User\Desktop\sql\chinook.db
Done.


Genre,Percent_contribution
Alternative,0.595%
Alternative & Punk,10.374%
Blues,2.593%
Bossa Nova,0.638%
Classical,1.743%
